## Check @docker-compose.yml  và @Dockerfile để tăng tốc quá trình build ?

Tôi sẽ đề xuất một số cải tiến cho cả docker-compose.yml và Dockerfile để tối ưu quá trình build:

1. Cải tiến Dockerfile:

````dockerfile:src/train_server/deployAPI_Server/Dockerfile
# Sử dụng image nhẹ hơn với python3.10-slim
FROM python:3.10-slim

# Thiết lập các biến môi trường để tối ưu Python và pip
ENV PYTHONUNBUFFERED=1 \
    PYTHONDONTWRITEBYTECODE=1 \
    PIP_NO_CACHE_DIR=1 \
    PIP_DISABLE_PIP_VERSION_CHECK=1

# Đặt thư mục làm việc
WORKDIR /app

# Cài đặt các package system cần thiết
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    && rm -rf /var/lib/apt/lists/*

# Sao chép và cài đặt requirements trước (để tận dụng cache)
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Sao chép mã nguồn
COPY ./api_export_v2_AgentOrWorkflow.py ./api_export.py

# Tạo non-root user để chạy ứng dụng
RUN adduser --disabled-password --gecos "" appuser
USER appuser

# Expose port
EXPOSE 25041

# Chạy ứng dụng với uvicorn
CMD ["uvicorn", "api_export:app", "--host", "0.0.0.0", "--port", "25041", "--workers", "4"]
````

2. Cải tiến docker-compose.yml:

````yaml:src/train_server/deployAPI_Server/docker-compose.yml
version: '3.8'

services:
  fastapi:
    build: 
      context: .
      dockerfile: Dockerfile
    container_name: deploy_FastResponseIntention
    ports:
      - "25041:25041"
    volumes:
      - ./trained_models:/app/trained_models:ro  # Thêm :ro để mount read-only
    environment:
      - MODEL_PATH=./trained_models/v7_trainsets_ckp-300_XLMRoBERTa_20eps
      - WORKERS=4
      - MAX_WORKERS=8
      - TIMEOUT=120
    deploy:
      resources:
        limits:
          cpus: '4'
          memory: 8G
        reservations:
          cpus: '2'
          memory: 4G
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:25041/docs"]
      interval: 30s
      timeout: 10s
      retries: 3
    restart: unless-stopped  # Thay đổi từ always sang unless-stopped
    logging:
      driver: "json-file"
      options:
        max-size: "10m"
        max-file: "3"
````

Các cải tiến chính:

### Dockerfile:
1. Sử dụng `python:3.10-slim` thay vì image FastAPI đầy đủ để giảm kích thước
2. Thêm các biến môi trường tối ưu cho Python và pip
3. Cài đặt các dependencies system cần thiết một cách tối ưu
4. Tạo non-root user để tăng bảo mật
5. Sử dụng multiple workers để tăng hiệu năng

### docker-compose.yml:
1. Thêm version để rõ ràng hơn
2. Mount volumes dưới dạng read-only
3. Thêm resource limits và reservations để quản lý tài nguyên
4. Thêm healthcheck để kiểm tra trạng thái service
5. Cấu hình logging để tránh log files quá lớn
6. Thêm các biến môi trường để điều chỉnh workers và timeout
7. Sử dụng `unless-stopped` thay vì `always` để tránh restart khi stop container có chủ ý

Các cải tiến này sẽ giúp:
- Giảm thời gian và kích thước build
- Tăng hiệu năng và độ ổn định
- Dễ dàng quản lý tài nguyên và monitoring
- Tăng tính bảo mật
- Tối ưu logging và debugging


Có vẻ như lỗi liên quan đến memory limit và swap. Tôi sẽ điều chỉnh lại cấu hình:

1. Sửa docker-compose.yml:

````yaml:src/train_server/deployAPI_Server/docker-compose.yml
version: '3.8'

services:
  fastapi:
    build: 
      context: .
      dockerfile: Dockerfile
    container_name: deploy_FastResponseIntention
    ports:
      - "25041:25041"
    volumes:
      - ./trained_models:/app/trained_models:ro
    environment:
      - MODEL_PATH=./trained_models/v7_trainsets_ckp-300_XLMRoBERTa_20eps
      - WORKERS=2  # Giảm số workers xuống
      - MAX_WORKERS=4  # Giảm max workers
      - TIMEOUT=120
    deploy:
      resources:
        limits:
          cpus: '2'  # Giảm CPU limit
          memory: 4G  # Giảm memory limit
        reservations:
          cpus: '1'
          memory: 2G
    # Bỏ healthcheck vì có thể gây vấn đề khi khởi động
    restart: unless-stopped
    logging:
      driver: "json-file"
      options:
        max-size: "10m"
        max-file: "3"
    # Thêm swap limit
    mem_swappiness: 0
    mem_reservation: 2G
````

2. Sửa Dockerfile:

````dockerfile:src/train_server/deployAPI_Server/Dockerfile
# Sử dụng image nhẹ hơn với python3.10-slim
FROM python:3.10-slim

# Thiết lập các biến môi trường để tối ưu Python và pip
ENV PYTHONUNBUFFERED=1 \
    PYTHONDONTWRITEBYTECODE=1 \
    PIP_NO_CACHE_DIR=1 \
    PIP_DISABLE_PIP_VERSION_CHECK=1

# Đặt thư mục làm việc
WORKDIR /app

# Cài đặt các package system cần thiết
RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    && rm -rf /var/lib/apt/lists/*

# Sao chép và cài đặt requirements trước
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Sao chép mã nguồn
COPY ./api_export_v2_AgentOrWorkflow.py ./api_export.py

# Sao chép mô hình
COPY ./trained_models/v7_trainsets_ckp-300_XLMRoBERTa_20eps ./trained_models/v7_trainsets_ckp-300_XLMRoBERTa_20eps

# Tạo non-root user
RUN adduser --disabled-password --gecos "" appuser
USER appuser

# Expose port
EXPOSE 25041

# Chạy ứng dụng với uvicorn và giới hạn memory
CMD ["uvicorn", "api_export:app", "--host", "0.0.0.0", "--port", "25041", "--workers", "2", "--limit-max-requests", "1000"]
````

Các thay đổi chính:
1. Giảm số lượng workers và giới hạn tài nguyên
2. Bỏ healthcheck tạm thời
3. Thêm cấu hình swap và memory reservation
4. Thêm limit-max-requests để tránh memory leak
5. Giảm số CPU và memory limits

Sau khi thay đổi, hãy thử:
```bash
# Dọn dẹp containers và images cũ
docker-compose down
docker system prune -f

# Khởi động lại
docker-compose up --build
```
